In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

In [2]:
path = '/home/hassan/lily/MLA/FDA/inpatient_simple_test.csv'
inpatient = pd.read_csv(path).iloc[:,1:]

inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
inpatient_date = inpatient_date.rename(columns = {'Discharge date':'Final discharge date'})
inpatient_date 

,Internalpatientid,Final discharge date
0,7,2013-04-18 14:41:31.0
1,9,2000-04-10 02:38:04.0
2,12,2004-04-07 03:40:16.0
3,17,2004-05-05 15:18:05.0
4,22,2014-05-07 15:43:10.0
...,...,...
22250,168995,2010-06-09 10:03:21.0
22251,169011,2004-07-15 15:11:27.0
22252,169037,2015-02-12 00:48:40.0
22253,169059,2003-03-11 12:21:08.0


In [5]:
# Use original inpatient date set to get admission date and discharge date
path = '/data/public/MLA/VCHAMPS-Test/inpatient_admissions_test.csv'
inpatient2 = pd.read_csv(path).iloc[:,1:]
inpatient2_date = inpatient2[['Internalpatientid','Admission date', 'Discharge date']]
inpatient2_date = inpatient2_date.sort_values(
    by = 'Internalpatientid',ascending=True).reset_index(drop = True)
inpatient2_date

,Internalpatientid,Admission date,Discharge date
0,7,2003-08-19 00:37:18.0,2003-08-19 23:32:28.0
1,7,2007-08-15 18:19:23.0,2007-08-26 16:28:46.0
2,7,2007-08-26 16:30:53.0,2007-09-23 17:14:50.0
3,7,2014-03-23 08:01:03.0,2014-03-25 11:20:29.0
4,7,2009-09-20 03:33:21.0,2009-09-22 09:26:12.0
...,...,...,...
139081,169065,1997-12-28 06:46:22.0,1997-12-29 07:14:14.0
139082,169065,2002-07-10 17:34:24.0,2002-07-24 19:50:33.0
139083,169065,2002-05-01 20:35:57.0,2002-05-06 19:27:54.0
139084,169065,2000-11-16 04:11:11.0,2000-11-25 07:38:10.0


In [6]:
# The last record of each patient will be consider as an
# indicator within or not within the next 300 days
# Filter out previous admissions before the last discharge record
# Consider them as input
inpatient_dates = inpatient2_date.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
inpatient_dates = inpatient_dates.loc[inpatient_dates['Discharge date'] <= 
                                       inpatient_dates['Final discharge date']]
inpatient_dates

,Internalpatientid,Admission date,Discharge date,Final discharge date
0,7,2003-08-19 00:37:18.0,2003-08-19 23:32:28.0,2013-04-18 14:41:31.0
1,7,2007-08-15 18:19:23.0,2007-08-26 16:28:46.0,2013-04-18 14:41:31.0
2,7,2007-08-26 16:30:53.0,2007-09-23 17:14:50.0,2013-04-18 14:41:31.0
4,7,2009-09-20 03:33:21.0,2009-09-22 09:26:12.0,2013-04-18 14:41:31.0
5,7,2013-04-16 08:48:18.0,2013-04-18 14:41:31.0,2013-04-18 14:41:31.0
...,...,...,...,...
139080,169065,2005-07-11 20:22:23.0,2005-07-19 23:03:27.0,2011-04-20 00:04:23.0
139081,169065,1997-12-28 06:46:22.0,1997-12-29 07:14:14.0,2011-04-20 00:04:23.0
139082,169065,2002-07-10 17:34:24.0,2002-07-24 19:50:33.0,2011-04-20 00:04:23.0
139083,169065,2002-05-01 20:35:57.0,2002-05-06 19:27:54.0,2011-04-20 00:04:23.0


In [7]:
path = '/data/public/MLA/VCHAMPS-Test/medications_ordered_test.csv'
medications = pd.read_csv(path).iloc[:,1:]
medications.head()

,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason,State
0,100,54.451524,2016-03-11 00:45:56.0,2016-03-11 10:57:56.0,2016-12-25 00:40:56.0,2016-12-25 00:40:56.0,hydrogen peroxide,discontinued,Requesting Physician Cancelled,New York
1,100,54.741514,2016-06-25 00:33:25.0,2016-06-24 09:39:25.0,2016-07-09 01:15:25.0,2016-07-09 01:15:25.0,guaifenesin,discontinued,Obsolete Order,New York
2,100,54.827014,2016-07-26 06:33:56.0,NaN,2016-12-26 05:34:56.0,2016-12-26 05:34:56.0,dextromethorphan ; guaifenesin,discontinued,Requesting Physician Cancelled,New York
3,100,55.089929,2016-10-30 08:51:18.0,2016-10-30 08:54:18.0,2016-10-31 09:36:18.0,2016-10-31 09:36:18.0,riluzole,discontinued,Requesting Physician Cancelled,New York
4,100,55.090329,2016-10-30 12:22:12.0,2016-10-30 12:22:12.0,2016-10-30 21:39:12.0,NaN,potassium chloride,expired,Not specified (no value),New York


In [8]:
medications['Order date'] = pd.to_datetime(medications['Order date'])
medications['Medication start date'] = pd.to_datetime(medications['Medication start date'])
medications['Medication stop date'] = pd.to_datetime(medications['Medication stop date'])
medications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15661661 entries, 0 to 15661660
Data columns (total 10 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Internalpatientid         int64         
 1   Age at med ordered        float64       
 2   Order date                datetime64[ns]
 3   Medication start date     datetime64[ns]
 4   Medication stop date      datetime64[ns]
 5   Order discontinue date    object        
 6   Ordered medication atc 5  object        
 7   Order status              object        
 8   Stop reason               object        
 9   State                     object        
dtypes: datetime64[ns](3), float64(1), int64(1), object(5)
memory usage: 1.2+ GB


In [9]:
medication_complete = medications.loc[medications['Order status'] == 'complete']
medication_complete = medication_complete.groupby('Internalpatientid').apply(lambda x:x).sort_values(by = ['Internalpatientid','Order date'],ascending=True).reset_index(drop = True)
medication_complete

/tmp/ipykernel_216075/2219256273.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  medication_complete = medication_complete.groupby('Internalpatientid').apply(lambda x:x).sort_values(by = ['Internalpatientid','Order date'],ascending=True).reset_index(drop = True)


,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason,State
0,6,79.393879,2006-01-29 11:06:34,NaT,2006-01-29 11:06:34,NaN,moxifloxacin,complete,Not specified (no value),Texas
1,6,80.342664,2007-01-11 05:50:47,NaT,2007-01-11 05:50:47,NaN,furosemide,complete,Not specified (no value),Texas
2,6,80.342664,2007-01-11 05:50:47,NaT,2007-01-11 05:50:47,NaN,lisinopril,complete,Not specified (no value),Texas
3,6,81.338677,2008-01-10 06:52:20,NaT,2008-01-10 06:57:20,NaN,azithromycin,complete,Not specified (no value),Texas
4,6,81.338677,2008-01-10 06:52:20,NaT,2008-01-10 06:57:20,NaN,guaifenesin,complete,Not specified (no value),Texas
...,...,...,...,...,...,...,...,...,...,...
268118,169065,53.266147,2011-05-22 18:38:35,2011-05-22 20:34:35,2011-05-23 11:49:35,NaN,aluminium hydroxide,complete,Not specified (no value),Arizona
268119,169065,53.268471,2011-05-23 15:01:45,2011-05-24 10:34:45,2011-05-26 03:34:45,NaN,diltiazem,complete,Not specified (no value),Arizona
268120,169065,53.282232,2011-05-28 15:44:17,2011-05-29 10:44:17,2011-05-30 15:46:17,NaN,vancomycin,complete,Not specified (no value),Arizona
268121,169065,53.293074,2011-06-01 14:50:31,NaT,2011-06-01 14:53:31,NaN,heparin,complete,Not specified (no value),Arizona


In [10]:
# Fill null values in start and stop date with order date
medication_complete['Medication start date'] = medication_complete['Medication start date'].fillna(medication_complete['Order date'])
medication_complete['Medication stop date'] = medication_complete['Medication stop date'].fillna(medication_complete['Order date'])

In [11]:
# Merge medication and inpatient table in order to 
# get number of medicaitons per admission
inpatient_med_complete = pd.merge(medication_complete, inpatient_dates, how = 'left', on = 'Internalpatientid')
inpatient_med_complete.head()

,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason,State,Admission date,Discharge date,Final discharge date
0,6,79.393879,2006-01-29 11:06:34,2006-01-29 11:06:34,2006-01-29 11:06:34,NaN,moxifloxacin,complete,Not specified (no value),Texas,NaN,NaN,NaN
1,6,80.342664,2007-01-11 05:50:47,2007-01-11 05:50:47,2007-01-11 05:50:47,NaN,furosemide,complete,Not specified (no value),Texas,NaN,NaN,NaN
2,6,80.342664,2007-01-11 05:50:47,2007-01-11 05:50:47,2007-01-11 05:50:47,NaN,lisinopril,complete,Not specified (no value),Texas,NaN,NaN,NaN
3,6,81.338677,2008-01-10 06:52:20,2008-01-10 06:52:20,2008-01-10 06:57:20,NaN,azithromycin,complete,Not specified (no value),Texas,NaN,NaN,NaN
4,6,81.338677,2008-01-10 06:52:20,2008-01-10 06:52:20,2008-01-10 06:57:20,NaN,guaifenesin,complete,Not specified (no value),Texas,NaN,NaN,NaN


In [12]:
# Make each medication order to the corresponding admission
inpatient_med_complete2 = inpatient_med_complete.loc[(inpatient_med_complete['Medication start date'] >= inpatient_med_complete['Admission date'])
                                                &(inpatient_med_complete['Medication start date'] <= inpatient_med_complete['Discharge date'])
                                                &(inpatient_med_complete['Medication stop date'] >= inpatient_med_complete['Admission date'])
                                                &(inpatient_med_complete['Medication stop date'] <= inpatient_med_complete['Discharge date'])]

In [13]:
inpatient_med_complete2 = inpatient_med_complete2.drop_duplicates(subset = ['Internalpatientid','Medication start date','Ordered medication atc 5'],keep = 'first')
inpatient_med_complete2 

,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason,State,Admission date,Discharge date,Final discharge date
19,12,69.912449,2000-05-10 00:00:19,2000-05-10 12:22:19,2000-05-13 20:38:19,NaN,acetylsalicylic acid,complete,Not specified (no value),Colorado,2000-05-09 15:42:38.0,2000-05-13 20:49:23.0,2004-04-07 03:40:16.0
30,12,71.931643,2002-05-18 00:22:32,2002-05-18 19:39:32,2002-05-20 02:19:32,NaN,acetylsalicylic acid,complete,Not specified (no value),Colorado,2002-05-18 00:04:03.0,2002-05-20 02:19:32.0,2004-04-07 03:40:16.0
35,12,71.932109,2002-05-18 04:27:45,2002-05-18 19:35:45,2002-05-20 02:15:45,NaN,gatifloxacin,complete,Not specified (no value),Colorado,2002-05-18 00:04:03.0,2002-05-20 02:19:32.0,2004-04-07 03:40:16.0
37,12,72.185439,2002-08-18 18:40:06,2002-08-19 10:21:06,2002-08-19 10:22:06,NaN,ceftriaxone,complete,Not specified (no value),Utah,2002-08-15 19:42:40.0,2002-08-26 10:54:34.0,2004-04-07 03:40:16.0
53,12,73.812482,2004-04-04 11:05:39,2004-04-05 10:13:39,2004-04-07 03:39:39,NaN,simvastatin,complete,Not specified (no value),Colorado,2004-04-04 10:59:02.0,2004-04-07 03:40:16.0,2004-04-07 03:40:16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3331933,168988,77.870232,2007-08-06 13:40:44,2007-08-07 11:29:44,2007-08-08 11:54:44,NaN,lisinopril,complete,Not specified (no value),Tennessee,2007-08-04 18:20:59.0,2007-08-09 15:18:03.0,2007-08-09 15:18:03.0
3331952,168995,62.261273,2008-03-24 11:53:12,2008-03-24 23:45:12,2008-03-25 10:42:12,NaN,pantoprazole,complete,Not specified (no value),Colorado,2008-03-24 11:26:58.0,2008-03-25 10:42:42.0,2010-06-09 10:03:21.0
3331954,168995,62.263136,2008-03-25 04:13:36,2008-03-25 09:39:36,2008-03-25 10:36:36,NaN,mometasone furoate,complete,Not specified (no value),Colorado,2008-03-24 11:26:58.0,2008-03-25 10:42:42.0,2010-06-09 10:03:21.0
3332022,169065,47.411619,2005-07-12 22:43:34,2005-07-13 06:39:34,2005-07-16 15:43:34,NaN,nitrofurantoin,complete,Not specified (no value),California,2005-07-11 20:22:23.0,2005-07-19 23:03:27.0,2011-04-20 00:04:23.0


In [14]:
# Calculate number of medication for each admission
tmp = pd.DataFrame(inpatient_med_complete2.groupby(['Internalpatientid', 'Admission date'])['Ordered medication atc 5'].count())
tmp = tmp.reset_index(drop = False)
tmp = tmp.rename(columns = {'Ordered medication atc 5':'Num medication per admission'})
tmp

,Internalpatientid,Admission date,Num medication per admission
0,12,2000-05-09 15:42:38.0,1
1,12,2002-05-18 00:04:03.0,2
2,12,2002-08-15 19:42:40.0,1
3,12,2004-04-04 10:59:02.0,4
4,17,2004-04-30 13:43:25.0,1
...,...,...,...
44934,168969,2010-02-15 21:48:22.0,1
44935,168988,2007-08-04 18:20:59.0,4
44936,168995,2008-03-24 11:26:58.0,2
44937,169065,2005-07-11 20:22:23.0,1


In [15]:
inpatient_med_complete3 = inpatient_med_complete2.merge(tmp, how = 'left', on = ['Internalpatientid', 'Admission date'])
inpatient_med_complete3['Total medications'] = inpatient_med_complete3['Num medication per admission'].groupby(inpatient_med_complete3['Internalpatientid']).transform('sum')
inpatient_med_complete3['Num med per admission mean'] = inpatient_med_complete3['Num medication per admission'].groupby(inpatient_med_complete3['Internalpatientid']).transform('mean')
inpatient_med_complete3['Num med per admission min'] = inpatient_med_complete3['Num medication per admission'].groupby(inpatient_med_complete3['Internalpatientid']).transform('min')
inpatient_med_complete3['Num med per admission max'] = inpatient_med_complete3['Num medication per admission'].groupby(inpatient_med_complete3['Internalpatientid']).transform('max')
inpatient_med_complete3

,Internalpatientid,Age at med ordered,Order date,Medication start date,Medication stop date,Order discontinue date,Ordered medication atc 5,Order status,Stop reason,State,Admission date,Discharge date,Final discharge date,Num medication per admission,Total medications,Num med per admission mean,Num med per admission min,Num med per admission max
0,12,69.912449,2000-05-10 00:00:19,2000-05-10 12:22:19,2000-05-13 20:38:19,NaN,acetylsalicylic acid,complete,Not specified (no value),Colorado,2000-05-09 15:42:38.0,2000-05-13 20:49:23.0,2004-04-07 03:40:16.0,1,22,2.75,1,4
1,12,71.931643,2002-05-18 00:22:32,2002-05-18 19:39:32,2002-05-20 02:19:32,NaN,acetylsalicylic acid,complete,Not specified (no value),Colorado,2002-05-18 00:04:03.0,2002-05-20 02:19:32.0,2004-04-07 03:40:16.0,2,22,2.75,1,4
2,12,71.932109,2002-05-18 04:27:45,2002-05-18 19:35:45,2002-05-20 02:15:45,NaN,gatifloxacin,complete,Not specified (no value),Colorado,2002-05-18 00:04:03.0,2002-05-20 02:19:32.0,2004-04-07 03:40:16.0,2,22,2.75,1,4
3,12,72.185439,2002-08-18 18:40:06,2002-08-19 10:21:06,2002-08-19 10:22:06,NaN,ceftriaxone,complete,Not specified (no value),Utah,2002-08-15 19:42:40.0,2002-08-26 10:54:34.0,2004-04-07 03:40:16.0,1,22,2.75,1,4
4,12,73.812482,2004-04-04 11:05:39,2004-04-05 10:13:39,2004-04-07 03:39:39,NaN,simvastatin,complete,Not specified (no value),Colorado,2004-04-04 10:59:02.0,2004-04-07 03:40:16.0,2004-04-07 03:40:16.0,4,22,2.75,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124858,168988,77.870232,2007-08-06 13:40:44,2007-08-07 11:29:44,2007-08-08 11:54:44,NaN,lisinopril,complete,Not specified (no value),Tennessee,2007-08-04 18:20:59.0,2007-08-09 15:18:03.0,2007-08-09 15:18:03.0,4,16,4.00,4,4
124859,168995,62.261273,2008-03-24 11:53:12,2008-03-24 23:45:12,2008-03-25 10:42:12,NaN,pantoprazole,complete,Not specified (no value),Colorado,2008-03-24 11:26:58.0,2008-03-25 10:42:42.0,2010-06-09 10:03:21.0,2,4,2.00,2,2
124860,168995,62.263136,2008-03-25 04:13:36,2008-03-25 09:39:36,2008-03-25 10:36:36,NaN,mometasone furoate,complete,Not specified (no value),Colorado,2008-03-24 11:26:58.0,2008-03-25 10:42:42.0,2010-06-09 10:03:21.0,2,4,2.00,2,2
124861,169065,47.411619,2005-07-12 22:43:34,2005-07-13 06:39:34,2005-07-16 15:43:34,NaN,nitrofurantoin,complete,Not specified (no value),California,2005-07-11 20:22:23.0,2005-07-19 23:03:27.0,2011-04-20 00:04:23.0,1,2,1.00,1,1


In [16]:
keep_columns = ['Internalpatientid','Num med per admission mean','Num med per admission min',
           'Num med per admission max','Total medications']

In [17]:
df = inpatient_med_complete3[keep_columns]
df = df.groupby('Internalpatientid').apply(lambda x:x)
df = df.drop_duplicates(keep = 'first').reset_index(drop = True)
df

/tmp/ipykernel_216075/2151990292.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Internalpatientid').apply(lambda x:x)


,Internalpatientid,Num med per admission mean,Num med per admission min,Num med per admission max,Total medications
0,12,2.750000,1,4,22
1,17,1.000000,1,1,1
2,25,3.000000,3,3,18
3,31,2.000000,2,2,4
4,41,3.800000,2,5,38
...,...,...,...,...,...
14693,168947,9.000000,9,9,81
14694,168969,6.153846,1,10,160
14695,168988,4.000000,4,4,16
14696,168995,2.000000,2,2,4


In [18]:
#df.to_csv('/home/daisy/FDA_Dataset/inpatient_medications_ordered_test_clean.csv')